
# First - get player_id       

url = http://mlb.mlb.com/pubajax/wf/flow/stats.splayer?season=2016&sort_order=%27desc%27&sort_column=%27avg%27&stat_type=hitting&page_type=SortablePlayer&team_id=112&game_type=%27R%27&player_pool=ALL&season_type=ANY&league_code=%27NL%27&sport_code=%27mlb%27&results=1000&recSP=1&recPP=50



# Second - use player_id to get gamelog

url = http://m.cubs.mlb.com/lookup/json/named.sport_hitting_game_log_composed.bam?game_type=%27R%27&league_list_id=%27mlb%27&player_id=592178&season=2016&sit_code=%271%27&sit_code=%272%27&sit_code=%273%27&sit_code=%274%27&sit_code=%275%27&sit_code=%276%27&sit_code=%277%27&sit_code=%278%27&sit_code=%279%27&sit_code=%2710%27&sit_code=%2711%27&sit_code=%2712%27

http://m.mlb.com/lookup/json/named.sport_hitting_game_log_composed.bam?game_type=%27R%27&league_list_id=%27mlb%27&player_id=493128&season=2016&sit_code=%271%27&sit_code=%272%27&sit_code=%273%27&sit_code=%274%27&sit_code=%275%27&sit_code=%276%27&sit_code=%277%27&sit_code=%278%27&sit_code=%279%27&sit_code=%2710%27&sit_code=%2711%27&sit_code=%2712%27


# need http://m.[team_name].mlb.com......  player_id=[player_id],season=[year],


In [9]:
import os
import csv
import requests as r
import json

def setSeasonAndTeamID(season,team_name,team_id):
    team_path = '../MLB_DATA/{}'.format(team_name)
    if not os.path.exists(team_path):
        os.makedirs(team_path)
    
    for season in range(season,1999,-1):
        getPlayerID(season,team_name,team_id,team_path)
def getPlayerID(season,team_name,team_id,team_path):
    resToGetPlayerID = r.get('http://mlb.mlb.com/pubajax/wf/flow/stats.splayer?season={}&sort_order=%27desc%27&sort_column=%27avg%27&stat_type=hitting&page_type=SortablePlayer&team_id={}&game_type=%27R%27&player_pool=ALL&season_type=ANY&league_code=%27NL%27&sport_code=%27mlb%27&results=1000&recSP=1&recPP=50'.format(season,team_id))
    playerStats = json.loads(resToGetPlayerID.text)['stats_sortable_player']['queryResults']['row']

    # for playerNameID in playerStat:
    #         print(playerNameID['name_display_first_last'])

    player_ids=[]
    playerIdName={}
    for playerID in playerStats:
        player_id = playerID['player_id']
        playerIdName[playerID['player_id']] = playerID['name_display_first_last']
        #print(player_ids)
        player_ids.append(player_id)

    for id in player_ids:
        getPlayerGameLog(id,playerIdName[id],season,team_name,team_path)
        
def getPlayerGameLog(id,playerName,season,team_name,team_path):
    resToGetGameLog = r.get('http://m.mlb.com/lookup/json/named.sport_hitting_game_log_composed.bam?game_type=%27R%27&league_list_id=%27mlb%27&player_id={}&season={}&sit_code=%271%27&sit_code=%272%27&sit_code=%273%27&sit_code=%274%27&sit_code=%275%27&sit_code=%276%27&sit_code=%277%27&sit_code=%278%27&sit_code=%279%27&sit_code=%2710%27&sit_code=%2711%27&sit_code=%2712%27'.format(id,season))
    gameLog = json.loads(resToGetGameLog.text)['sport_hitting_game_log_composed']['sport_hitting_game_log']['queryResults']['row']
    #gameSize = json.loads(resToGetGameLog.text)['sport_hitting_game_log_composed']['sport_hitting_game_log']['queryResults']['totalSize']

    season_path = team_path + '/{}'.format(season)
    if not os.path.exists(season_path):
        os.makedirs(season_path)
    try:
        fieldnames = list((gameLog[0]).keys())
        with open(season_path + '/{}.csv'.format(playerName),'w') as csvfile:

            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()

            for row in gameLog:
                writer.writerow(row)
    except KeyError:
        with open(team_path + '/except_player.txt','w') as text_file:
            print(team_name +':'+ str(season) + ':' + id + ' : ' + playerName)
            text_file.write(team_name +':'+ str(season) + ':' + id + ':' + playerName)


In [10]:


#NLCentral = ['112','138','134','158','113']  #Chicago Cubs,St. Louis Cardinals,Pittsburgh Pirates,Milwaukee Brewers,Cincinnati Reds
NLCentral = {'112':'Chicago Cubs',
             '138':'St. Louis Cardinals',
             '134':'Pittsburgh Pirates',
             '158':'Milwaukee Brewers',
             '113':'Cincinnati Reds'
            }

for team_id in NLCentral:
    setSeasonAndTeamID(2016,NLCentral[team_id],team_id) #setSeasonAndTeamID(season,team_name,team_id) #season max value=2016 
            

Milwaukee Brewers:2016:516414 : Ariel Pena
Milwaukee Brewers:2014:431166 : Matt Pagnozzi
Milwaukee Brewers:2011:407576 : Mike Rivera
Milwaukee Brewers:2011:456544 : Eric Farris
Milwaukee Brewers:2009:449786 : Carlos Corporan
Milwaukee Brewers:2009:228594 : David Riske
Milwaukee Brewers:2008:457130 : Vinny Rottino
Milwaukee Brewers:2008:435638 : Zach Jackson
Milwaukee Brewers:2006:430834 : Chris Mabeus
Milwaukee Brewers:2005:119442 : Julio Mosquera
Milwaukee Brewers:2004:430611 : Corey Hart
Milwaukee Brewers:2004:430609 : Chris Saenz
Milwaukee Brewers:2001:136304 : Valerio De Los Santos


ConnectionError: HTTPConnectionPool(host='mlb.mlb.com', port=80): Max retries exceeded with url: /pubajax/wf/flow/stats.splayer?season=2016&sort_order=%27desc%27&sort_column=%27avg%27&stat_type=hitting&page_type=SortablePlayer&team_id=134&game_type=%27R%27&player_pool=ALL&season_type=ANY&league_code=%27NL%27&sport_code=%27mlb%27&results=1000&recSP=1&recPP=50 (Caused by NewConnectionError('<requests.packages.urllib3.connection.HTTPConnection object at 0x0000000005868D68>: Failed to establish a new connection: [Errno 11004] getaddrinfo failed',))